In [1]:
import torch
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd
import ast
from Bio import SeqIO
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/homebrew/anaconda3/envs/AIMA05/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/homebrew/anaconda3/envs/AIMA05/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/homebrew/anaconda3/envs/AIMA05/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/anaconda3/envs/AIMA05/lib/python3.10/site-packages/traitlets/config/application.py", lin

# 1. Data preparation

In [2]:
df_proteins = pd.read_parquet("data_preparation/all_proteins_with_fold.parquet")
print(df_proteins.shape)
display(df_proteins.head())

(1169, 13)


,id,sequence,seq_len,binding_sites,num_residues,target,sites_pos_min,sites_pos_max,sites_pos_mean,sites_pos_std,sites_pos_median,is_duplicate,fold
0,Q5LL55,MSETWLPTLVTATPQEGFDLAVKLSRIAVKKTQPDAQVRDTLRAVY...,76,"[53, 46, 32, 42, 25, 60, 56, 43, 57, 31, 54, 2...",17,small,22,60,40.529412,11.887302,40,False,1
1,H9L4N9,MQINIQGHHIDLTDSMQDYVHSKFDKLERFFDHINHVQVILRVEKL...,95,"[51, 62, 42, 60, 55, 64]",6,metal,42,64,55.666667,7.498148,60,False,1
2,O34738,MKSWKVKEIVIMSVISIVFAVVYLLFTHFGNVLAGMFGPIAYEPIY...,199,"[100, 46, 104, 88, 63, 42, 77, 135, 91, 108, 2...",22,small,23,135,86.227273,34.010602,95,False,1
3,P39579,MDFKQEVLDVLAEVCQDDIVKENPDIEIFEEGLLDSFGTVELLLAI...,78,"[37, 36, 40, 61]",4,small,36,61,43.500000,10.210289,40,False,1
4,P01887,MARSVTLVFLVLVSLTGLYAIQKTPQIQVYSRHPPENGKPNILNCY...,119,"[83, 77]",2,metal,77,83,80.000000,3.000000,83,True,1


In [3]:
# check sample in the dataset with min sequence length
min_seq_len = df_proteins["seq_len"].min()
min_idx = df_proteins[df_proteins["seq_len"] == min_seq_len].index[0]

# print sample sequence and binding sites
print("Sequence:", df_proteins["sequence"].loc[min_idx])
print("Binding sites:", df_proteins["binding_sites"].loc[min_idx])

Sequence: MLAIVAYIGFLALFTGIAAGLLFGLRSAKIL
Binding sites: [ 8 12 16  3 23 15  4  7]


In [4]:
# print min, max sequence length
min_seq_len = df_proteins["seq_len"].min()
max_seq_len = df_proteins["seq_len"].max()
print("Min sequence length:", min_seq_len)
print("Max sequence length:", max_seq_len)

Min sequence length: 31
Max sequence length: 556


## 1.1.Create vocabulary

In [5]:
def antigen_tokenizer(antigen):
    return [aa for aa in antigen]


def create_vocab(data, vocab_size):
    # Create a function to yield list of tokens
    def yield_tokens(antigens):
        for seq in antigens:
            yield antigen_tokenizer(seq)

    # Create vocabulary
    vocab = build_vocab_from_iterator(
        yield_tokens(data), max_tokens=vocab_size, specials=["<pad>", "<unk>"]
    )
    vocab.set_default_index(vocab["<unk>"])
    return vocab

In [6]:
vocab = create_vocab(df_proteins["sequence"], 30)
dict(sorted(vocab.get_stoi().items(), key=lambda item: item[1]))

{'<pad>': 0,
 '<unk>': 1,
 'L': 2,
 'A': 3,
 'E': 4,
 'G': 5,
 'V': 6,
 'S': 7,
 'K': 8,
 'D': 9,
 'I': 10,
 'T': 11,
 'R': 12,
 'P': 13,
 'N': 14,
 'F': 15,
 'Q': 16,
 'Y': 17,
 'M': 18,
 'H': 19,
 'C': 20,
 'W': 21,
 'U': 22,
 'X': 23}

## 1.2.Create dataset

In [7]:
class EpitopeDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [22]:
def vectorize_amino_acids(antigen, vocab, max_len):
    encoded_seq = vocab(antigen_tokenizer(antigen))
    num_pads = max_len - len(antigen)
    encoded_seq += [vocab["<pad>"]] * num_pads
    return encoded_seq


def prepare_dataset(df, vocab, max_len):
    # encoding aminio acids
    X = [vectorize_amino_acids(seq, vocab, max_len) for seq in df["sequence"]]

    # residues to one-hot
    n_seq = df.shape[0]
    Y = np.zeros((n_seq, max_len))
    for i, labels in enumerate(df["binding_sites"]):
        for pos in labels:
            # -1 because the first position in array is 0
            Y[i, pos - 1] = 1

    return EpitopeDataset(X, Y)

In [23]:
# split into different folds based on column "fold"
n_folds = 5
fold_datasets = []
for fold in range(n_folds):
    dataset = prepare_dataset(
        df=df_proteins[df_proteins["fold"] == fold + 1],
        vocab=vocab,
        max_len=max_seq_len,
    )
    fold_datasets.append(dataset)

# check the number of samples in each fold
for i, fold_data in enumerate(fold_datasets):
    print(f"Fold {i}: {len(fold_data)} samples")

Fold 0: 234 samples
Fold 1: 229 samples
Fold 2: 232 samples
Fold 3: 236 samples
Fold 4: 238 samples


In [24]:
# check sample in the dataset with min sequence length
fold1_dataset = fold_datasets[0]
X_i, y_i = fold1_dataset[0]
print("X shape:", len(X_i))
print("X:", X_i[:30])
print("Y shape:", y_i.shape)
print("Y:", y_i[:30])

X shape: 556
X: [18, 7, 4, 11, 21, 2, 13, 11, 2, 6, 11, 3, 11, 13, 16, 4, 5, 15, 9, 2, 3, 6, 8, 2, 7, 12, 10, 3, 6, 8]
Y shape: (556,)
Y: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 1. 0.]


In [25]:
seq_length = max_seq_len


def collate_batch(batch):
    # create inputs, offsets, labels for batch
    sentences, labels = list(zip(*batch))
    encoded_sentences = [
        (
            sentence + ([0] * (seq_length - len(sentence)))
            if len(sentence) < seq_length
            else sentence[:seq_length]
        )
        for sentence in sentences
    ]

    encoded_sentences = torch.tensor(encoded_sentences, dtype=torch.int64)
    labels = torch.tensor(labels)

    return encoded_sentences, labels

In [26]:
batch_size = 128

train_dataloader = DataLoader(
    fold_datasets[0], batch_size=batch_size, shuffle=True, collate_fn=collate_batch
)

valid_dataloader = DataLoader(
    fold_datasets[1], batch_size=batch_size, shuffle=False, collate_fn=collate_batch
)

test_dataloader = DataLoader(
    fold_datasets[2], batch_size=batch_size, shuffle=False, collate_fn=collate_batch
)

In [29]:
encoded_sentences, labels = next(iter(train_dataloader))
print("Encoded sentences shape:", encoded_sentences.shape)
print("Labels shape:", labels.shape)

Encoded sentences shape: torch.Size([128, 556])
Labels shape: torch.Size([128, 556])


# 2. Model Definition

In [40]:
class RNNEpitopeModel(nn.Module):
    def __init__(
        self,
        vocab_size=21,
        embedding_dim=4,
        hidden_size=3,
        activation="relu",
        num_classes=2,
    ):
        super(RNNEpitopeModel, self).__init__()

        # define embedding layers
        self.embedding_dim = embedding_dim
        self.embedding_layer = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_dim
        )

        # define RNN layer
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=1,
            nonlinearity=activation,
            batch_first=True,
        )
        self.fc = nn.Linear(self.hidden_size, num_classes)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        rnn_output, _ = self.rnn(embeddings)
        logits = self.fc(rnn_output)
        output = F.sigmoid(logits)
        return output

In [41]:
model = RNNEpitopeModel(vocab_size=len(vocab))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
model = model.to(device)
model

Device: cpu


RNNEpitopeModel(
  (embedding_layer): Embedding(24, 4)
  (rnn): RNN(4, 3, batch_first=True)
  (fc): Linear(in_features=3, out_features=2, bias=True)
)

In [43]:
encoded_sentences = encoded_sentences.to(device)
predictions = model(encoded_sentences.to(device))
predictions.shape

torch.Size([128, 556, 2])

# 3. Training and Evaluation

In [51]:
def train_epoch(
    model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50
):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        predictions = model(inputs)
        print(predictions.shape)
        print(labels.shape)

        # compute loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # backward
        loss.backward()
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(train_dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [52]:
def evaluate_epoch(model, criterion, valid_dataloader, device):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)

            loss = criterion(predictions, labels)
            losses.append(loss.item())

            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [53]:
def train(
    model,
    model_name,
    save_model,
    optimizer,
    criterion,
    train_dataloader,
    valid_dataloader,
    num_epochs,
    device,
):
    train_accs, train_losses = [], []
    eval_accs, eval_losses = [], []
    best_loss_eval = 100
    times = []
    for epoch in range(1, num_epochs + 1):
        epoch_start_time = time.time()
        # Training
        train_acc, train_loss = train_epoch(
            model, optimizer, criterion, train_dataloader, device, epoch
        )
        train_accs.append(train_acc)
        train_losses.append(train_loss)

        # Evaluation
        eval_acc, eval_loss = evaluate_epoch(model, criterion, valid_dataloader, device)
        eval_accs.append(eval_acc)
        eval_losses.append(eval_loss)

        # Save best model
        if eval_loss < best_loss_eval:
            torch.save(model.state_dict(), save_model + f"/{model_name}.pt")

        times.append(time.time() - epoch_start_time)
        # Print loss, acc end epoch
        print("-" * 59)
        print(
            "| End of epoch {:3d} | Time: {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3f} "
            "| Valid Accuracy {:8.3f} | Valid Loss {:8.3f} ".format(
                epoch,
                time.time() - epoch_start_time,
                train_acc,
                train_loss,
                eval_acc,
                eval_loss,
            )
        )
        print("-" * 59)

    # Load best model
    model.load_state_dict(
        torch.load(save_model + f"/{model_name}.pt", weights_only=True)
    )
    model.eval()
    metrics = {
        "train_accuracy": train_accs,
        "train_loss": train_losses,
        "valid_accuracy": eval_accs,
        "valid_loss": eval_losses,
        "time": times,
    }
    return model, metrics

In [54]:
import matplotlib.pyplot as plt


def plot_result(num_epochs, train_accs, eval_accs, train_losses, eval_losses):
    epochs = list(range(num_epochs))
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    axs[0].plot(epochs, train_accs, label="Training")
    axs[0].plot(epochs, eval_accs, label="Evaluation")
    axs[1].plot(epochs, train_losses, label="Training")
    axs[1].plot(epochs, eval_losses, label="Evaluation")
    axs[0].set_xlabel("Epochs")
    axs[1].set_xlabel("Epochs")
    axs[0].set_ylabel("Accuracy")
    axs[1].set_ylabel("Loss")
    plt.legend()

In [ ]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

model = RNNEpitopeModel(vocab_size=len(vocab))
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 50
save_model = "./model"
os.makedirs(save_model, exist_ok=True)
model_name = "model"

model, metrics = train(
    model,
    model_name,
    save_model,
    optimizer,
    criterion,
    train_dataloader,
    valid_dataloader,
    num_epochs,
    device,
)